In [220]:
from langchain_groq import ChatGroq

In [221]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key=' use your api key here ', 
    model_name="llama3-70b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to set foot on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the moon.


In [222]:
from langchain_community.document_loaders import WebBaseLoader
import textwrap

# Load the page content first
loader = WebBaseLoader("https://careers.capgemini.com/job/Pune-Software-Engineer/1182147201/")
page_data = loader.load().pop().page_content  # Ensure page_data is assigned before using it

# Now use page_data for text wrapping
wrapped_lines = textwrap.wrap(page_data, width=150)
for line in wrapped_lines:
    print(line)




           Software Engineer Job Details | Capgemini                         We use cookies to offer you the best possible website experience. Your
cookie preferences will be stored in your browser’s local storage. This includes cookies necessary for the website's operation. Additionally, you can
freely decide and change any time whether you accept cookies or choose to opt out of cookies to improve the website's performance, as well as cookies
used to display content tailored to your interests. Your experience of the site and the services we are able to offer may be impacted if you do not
accept all cookies.                  Modify Cookie Preferences                       Reject Optional Cookies                  Accept All Cookies
Press Tab to Move to Skip to Content Link Skip to main content             View Profile     Employee Login                             View Profile
Employee Login                             View Profile     Employee Login                                 Sear

In [223]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [224]:


import json
import textwrap
from langchain_core.output_parsers import JsonOutputParser

# Initialize parser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

# Ensure json_res is a dictionary, not a list
if isinstance(json_res, list) and len(json_res) == 1:
    json_res = json_res[0]  # Convert list to dictionary

# Print formatted output
print(f"Role: {json_res['role']}")
print(f"Experience: {json_res['experience']}")
print(f"Skills: {', '.join(json_res['skills'])}")

# Wrap and print description properly
print("Description:")
wrapped_description = textwrap.wrap(json_res['description'], width=80)  # Adjust width
for line in wrapped_description:
    print(line)  # Prints each line separately for proper formatting





Role: Software Engineer
Experience: Experienced Professionals
Skills: Verbal Communication
Description:
Works in the area of Software Engineering, which encompasses the development,
maintenance and optimization of software solutions/applications. Applies
scientific methods to analyse and solve software engineering problems. He/she is
responsible for the development and application of software engineering practice
and knowledge, in research, design, development and maintenance. His/her work
requires the exercise of original thought and judgement and the ability to
supervise the technical and administrative work of other software engineers. The
software engineer builds skills and expertise of his/her software engineering
discipline to reach standard software engineer skills expectations for the
applicable role, as defined in Professional Communities. The software engineer
collaborates and acts as team player with other software engineers and
stakeholders.


In [225]:
type(json_res)



dict

In [226]:
import pandas as pd

df = pd.read_csv("C:/Users/pasun/OneDrive/Desktop/New folder/my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [227]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [228]:
'''links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links'''
# Run the query
query_result = collection.query(query_texts=job['skills'], n_results=2)

# Debugging: Check what type query_result is
print(type(query_result))  # Check if it's a list or dict
print(query_result)        # Print the actual output

# If query_result is a list, access its elements properly
if isinstance(query_result, list):
    # Extract 'metadatas' from each dictionary inside the list
    links = [item.get('metadatas', []) for item in query_result]
else:
    # If it's a dictionary, get 'metadatas' directly
    links = query_result.get('metadatas', [])

# Print final output
print(links)



<class 'dict'>
{'ids': [['933d725a-ecd4-48e7-a693-5d462f5dd836', '44a51b61-e656-42a7-a47e-6a19e8881605']], 'embeddings': None, 'documents': [['Android, Java, Room Persistence', 'Machine Learning, Python, TensorFlow']], 'uris': None, 'data': None, 'metadatas': [[{'links': 'https://example.com/android-portfolio'}, {'links': 'https://example.com/ml-python-portfolio'}]], 'distances': [[1.8089731191611407, 1.8773474825781715]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
[[{'links': 'https://example.com/android-portfolio'}, {'links': 'https://example.com/ml-python-portfolio'}]]


In [229]:
job

{'role': 'Software Engineer',
 'experience': 'Experienced Professionals',
 'skills': ['Verbal Communication'],
 'description': 'Works in the area of Software Engineering, which encompasses the development, maintenance and optimization of software solutions/applications. Applies scientific methods to analyse and solve software engineering problems. He/she is responsible for the development and application of software engineering practice and knowledge, in research, design, development and maintenance. His/her work requires the exercise of original thought and judgement and the ability to supervise the technical and administrative work of other software engineers. The software engineer builds skills and expertise of his/her software engineering discipline to reach standard software engineer skills expectations for the applicable role, as defined in Professional Communities. The software engineer collaborates and acts as team player with other software engineers and stakeholders.'}

In [230]:
job = json_res
job['skills']

['Verbal Communication']

In [231]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Here is the cold email:

Subject: Expert Software Engineering Solutions for Your Organization

Dear [Hiring Manager's Name],

I came across your job posting for a Software Engineer and was impressed by the scope of the role. As a Business Development Executive at AtliQ, I believe our team of experts can help you fulfill your software engineering needs.

At AtliQ, we specialize in developing, maintaining, and optimizing software solutions that cater to the unique requirements of our clients. Our team of seasoned software engineers applies scientific methods to analyze and solve complex software engineering problems, ensuring the delivery of high-quality solutions that meet the highest standards.

Our expertise in software engineering is backed by our successful projects in Android app development (https://example.com/android-portfolio) and machine learning with Python (https://example.com/ml-python-portfolio). We have empowered numerous enterprises with tailored solutions, resulting in 